In [1]:
from prophet import Prophet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import datetime
import warnings

warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('crime_theft.csv', encoding='cp949', index_col=0)
df.head()

,20년1월_절도,20년2월_절도,20년3월_절도,20년4월_절도,20년5월_절도,20년6월_절도,20년7월_절도,20년8월_절도,20년9월_절도,20년10월_절도,...,22년3월_절도,22년4월_절도,22년5월_절도,22년6월_절도,22년7월_절도,22년8월_절도,22년9월_절도,22년10월_절도,22년11월_절도,22년12월_절도
자치구,,,,,,,,,,,,,,,,,,,,,
강남구,310,240,279,227,215,234,286,269,121,200,...,161,200,228,228,222,247,168,194,313,234
강동구,146,132,137,124,127,132,149,160,44,98,...,96,104,141,136,203,158,126,107,236,122
강북구,79,61,72,66,67,79,86,61,91,62,...,65,62,101,81,67,86,73,71,84,67
강서구,193,161,179,116,150,121,121,136,54,134,...,130,128,155,208,198,193,164,214,200,170
관악구,223,181,164,202,207,179,208,158,80,187,...,124,170,140,167,170,215,95,142,290,202


In [3]:
df.columns

Index(['20년1월_절도', '20년2월_절도', '20년3월_절도', '20년4월_절도', '20년5월_절도', '20년6월_절도',
       '20년7월_절도', '20년8월_절도', '20년9월_절도', '20년10월_절도', '20년11월_절도',
       '20년12월_절도', '21년1월_절도', '21년2월_절도', '21년3월_절도', '21년4월_절도', '21년5월_절도',
       '21년6월_절도', '21년7월_절도', '21년8월_절도', '21년9월_절도', '21년10월_절도',
       '21년11월_절도', '21년12월_절도', '22년1월_절도', '22년2월_절도', '22년3월_절도',
       '22년4월_절도', '22년5월_절도', '22년6월_절도', '22년7월_절도', '22년8월_절도', '22년9월_절도',
       '22년10월_절도', '22년11월_절도', '22년12월_절도'],
      dtype='object')

In [4]:
pattern = ['20년', '21년', '22년']
months = [f'{month}월' for month in range(1, 13)]
sorted_columns = [f'{year}{month}_절도' for year in pattern for month in months]

# 패턴에 따라 칼럼 이름 정렬
df = df.reindex(columns=sorted_columns)

In [5]:
df.head()

,20년1월_절도,20년2월_절도,20년3월_절도,20년4월_절도,20년5월_절도,20년6월_절도,20년7월_절도,20년8월_절도,20년9월_절도,20년10월_절도,...,22년3월_절도,22년4월_절도,22년5월_절도,22년6월_절도,22년7월_절도,22년8월_절도,22년9월_절도,22년10월_절도,22년11월_절도,22년12월_절도
자치구,,,,,,,,,,,,,,,,,,,,,
강남구,310,240,279,227,215,234,286,269,121,200,...,161,200,228,228,222,247,168,194,313,234
강동구,146,132,137,124,127,132,149,160,44,98,...,96,104,141,136,203,158,126,107,236,122
강북구,79,61,72,66,67,79,86,61,91,62,...,65,62,101,81,67,86,73,71,84,67
강서구,193,161,179,116,150,121,121,136,54,134,...,130,128,155,208,198,193,164,214,200,170
관악구,223,181,164,202,207,179,208,158,80,187,...,124,170,140,167,170,215,95,142,290,202


In [6]:
data = df.T
data.to_csv('시계열분석_전처리_절도.csv', encoding='cp949')
data.head()

자치구,강남구,강동구,강북구,강서구,관악구,광진구,구로구,금천구,노원구,도봉구,...,성동구,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구
20년1월_절도,310,146,79,193,223,175,170,79,101,81,...,95,80,250,130,203,104,126,118,216,112
20년2월_절도,240,132,61,161,181,134,100,107,82,61,...,66,57,148,141,173,70,93,87,135,117
20년3월_절도,279,137,72,179,164,145,128,97,83,59,...,75,93,231,134,193,108,104,135,176,140
20년4월_절도,227,124,66,116,202,126,159,72,110,61,...,83,81,204,114,177,116,97,81,118,104
20년5월_절도,215,127,67,150,207,130,163,77,151,63,...,78,76,242,123,198,105,88,85,117,105


In [111]:
df = pd.read_csv('시계열분석_전처리_절도.csv', encoding='cp949')
df.head()

,Date,강남구,강동구,강북구,강서구,관악구,광진구,구로구,금천구,노원구,...,성동구,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구
0,20년1월_절도,310,146,79,193,223,175,170,79,101,...,95,80,250,130,203,104,126,118,216,112
1,20년2월_절도,240,132,61,161,181,134,100,107,82,...,66,57,148,141,173,70,93,87,135,117
2,20년3월_절도,279,137,72,179,164,145,128,97,83,...,75,93,231,134,193,108,104,135,176,140
3,20년4월_절도,227,124,66,116,202,126,159,72,110,...,83,81,204,114,177,116,97,81,118,104
4,20년5월_절도,215,127,67,150,207,130,163,77,151,...,78,76,242,123,198,105,88,85,117,105


In [112]:
# '범죄건수' 문자열 삭제
df['Date'] = df['Date'].str.replace('_절도', '')

df['Date'] = df['Date'].str.replace(r'(\d+)년(\d+)월', r'20\1\2', regex=True)

df['Date'] = pd.to_datetime(df['Date'], format='%Y%m').dt.strftime('%Y%m')

df.head()

,Date,강남구,강동구,강북구,강서구,관악구,광진구,구로구,금천구,노원구,...,성동구,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구
0,202001,310,146,79,193,223,175,170,79,101,...,95,80,250,130,203,104,126,118,216,112
1,202002,240,132,61,161,181,134,100,107,82,...,66,57,148,141,173,70,93,87,135,117
2,202003,279,137,72,179,164,145,128,97,83,...,75,93,231,134,193,108,104,135,176,140
3,202004,227,124,66,116,202,126,159,72,110,...,83,81,204,114,177,116,97,81,118,104
4,202005,215,127,67,150,207,130,163,77,151,...,78,76,242,123,198,105,88,85,117,105


In [113]:
gu_name = ['중랑구', 'Date']

last_3year = list()
for i in range(2023, 2027):
    for j in range(1, 13):
        last_3year.append(['%04d-%02d' % (i,j)])
        
        
last_3year = pd.DataFrame(last_3year, columns = ['ds'])
last_3year['ds']= pd.to_datetime(last_3year['ds'])

for i in range(len(df['Date'])):
    date_time_str = df['Date'][i]
    
    date = datetime.datetime.strptime(str(date_time_str), '%Y%m')  #datetime.datetime.strptime(date, "%Y-%m-%d")
    
    df['Date'][i] = date

gangdong_predict = pd.DataFrame()

for i,name in enumerate(gu_name):
    data = pd.DataFrame()
    data = df[[name,'Date']]
    data.columns = ['y','ds']
    data['ds'] = pd.to_datetime(data['ds'])

    model = Prophet().fit(data)
    print(i,"model_fit")
    forecast = model.predict(last_3year)
    print(i,"model_predict")
    gangdong_predict = pd.concat([gangdong_predict, forecast['yhat']], ignore_index = True, axis=1)

15:38:09 - cmdstanpy - INFO - Chain [1] start processing
15:38:09 - cmdstanpy - INFO - Chain [1] done processing


0 model_fit
0 model_predict


TypeError: Invalid object type at position 0

In [114]:
gangdong_predict = pd.concat([gangdong_predict, forecast['ds']], ignore_index = True, axis=1)

gangdong_predict.columns = ['중랑구', 'Date']

gangdong_predict.to_csv('jungrang_theft_predict.csv', encoding='cp949',index = False)